In [ ]:
! pip install rasterio shapely pyproj geopandas

In [ ]:
import rasterio
from rasterio.mask import mask
from rasterio.warp import transform_geom
import json
from shapely.geometry import shape
import os
from datetime import datetime, timedelta
import numpy as np
from pyproj import Transformer
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# Constants

In [ ]:
polygon_file = "/content/Kocaeli2024Data-request.json"
data_folder = "/content/2024data"
MODIS_PARAMS = {
    "MCD12Q1.061": {
        "scale": 1,
        "offset": 0,
        "unit_conversion": lambda x: x,  # Categorical
        "nodata_mask": lambda data: (data == 0) | (data == 255),
        "output_name": "LandCover_Type",
    },
    "MOD11A2.061": {
        "scale": 0.02,
        "offset": 0,
        "unit_conversion": lambda k: k - 273.15,  # Kelvin to Celsius
        "nodata_mask": lambda data: (data <= 7500) | (data > 65535),
        "output_name": "LST_Celsius",
    },
    "MOD13A2.061": {
        "scale": 0.0001,
        "offset": 0,
        "unit_conversion": lambda x: x,
        "nodata_mask": lambda data: data <= -2000,
        "output_name": "NDVI_Value",
    },
    "MOD17A3HGF.061": {
        "scale": 0.0001,
        "offset": 0,
        "unit_conversion": lambda x: x,
        "nodata_mask": lambda data: data <= -3000,
        "output_name": "NPP_kgCm2perYear",
    },
    "MOD44B.061": {
        "scale": 1,
        "offset": 0,
        "unit_conversion": lambda x: x,
        "nodata_mask": lambda data: (data == 200) | (data == 255),
        "output_name": "TreeCover_Percent",
    },
}

# Polygon Load

In [ ]:
print("Loading polygon from JSON...")
with open(polygon_file, "r") as f:
    data = json.load(f)
    features = data["params"]["geo"]["features"]
    polygon_geojson = features[0]["geometry"]
    polygon = shape(polygon_geojson)

Loading polygon from JSON...


# Data Init

In [ ]:
data_files = []
import os
for file in os.listdir(data_folder):
    if file.endswith(".tif"):
        data_files.append(os.path.join(data_folder, file))

quality_files = [i for i in data_files if "QC" in i]
data_files = [x for x in data_files if x not in quality_files]

print(data_files)
print(quality_files)

['/content/2024data/MOD11A2.061_LST_Day_1km_doy2024361000000_aid0001.tif', '/content/2024data/MOD13A2.061__1_km_16_days_NDVI_doy2024241000000_aid0001.tif', '/content/2024data/MOD16A2GF.061_ET_500m_doy2024273000000_aid0001.tif', '/content/2024data/MOD17A3HGF.061_Npp_500m_doy2024001000000_aid0001.tif', '/content/2024data/MOD11A2.061_LST_Night_1km_doy2024281000000_aid0001.tif', '/content/2024data/MOD11A2.061_LST_Day_1km_doy2024185000000_aid0001.tif', '/content/2024data/MOD16A2GF.061_ET_500m_doy2024089000000_aid0001.tif', '/content/2024data/MOD13A2.061__1_km_16_days_VI_Quality_doy2024001000000_aid0001.tif', '/content/2024data/MOD11A2.061_LST_Day_1km_doy2024153000000_aid0001.tif', '/content/2024data/MOD13A2.061__1_km_16_days_NDVI_doy2024289000000_aid0001.tif', '/content/2024data/MOD11A2.061_LST_Night_1km_doy2023361000000_aid0001.tif', '/content/2024data/MOD11A2.061_LST_Night_1km_doy2024169000000_aid0001.tif', '/content/2024data/MOD11A2.061_LST_Day_1km_doy2024233000000_aid0001.tif', '/conten

# Data Load

In [ ]:
def load_modis_tif(filepath, catalogue, date):
    params = MODIS_PARAMS[catalogue]
    output_name = params["output_name"]

    with rasterio.open(filepath) as src:
        data = src.read(1)
        transform = src.transform
        height, width = data.shape

        cols, rows = np.meshgrid(np.arange(width), np.arange(height))
        lon, lat = rasterio.transform.xy(transform, rows, cols)
        lon = np.array(lon)
        lat = np.array(lat)

        data_scaled = data * params["scale"] + params["offset"]
        data_converted = params["unit_conversion"](data_scaled)
        nodata_mask = params["nodata_mask"](data)

        lon_flat = lon.flatten()
        lat_flat = lat.flatten()
        data_flat = data_converted.flatten()
        mask_flat = nodata_mask.flatten()

        valid_idx = ~mask_flat

        df = pd.DataFrame({
            'date': date,
            'longitude': lon_flat[valid_idx],
            'latitude': lat_flat[valid_idx],
            output_name: data_flat[valid_idx]
        })

        return df

In [ ]:
all_landcover_dfs = []
all_lst_dfs = []
all_ndvi_dfs = []
all_npp_dfs = []
all_treecover_dfs = []
all_dfs = {}

In [ ]:
data_dict = {}

for data_path in data_files:
  parsed = data_path.replace("/content/2024data/", "")
  parsed = parsed.split("_", maxsplit=1)
  catalogue = parsed[0]

  rdat = parsed[1].rsplit("_", maxsplit=2)
  layer, date = rdat[0], rdat[1]

  date = date.replace("doy", "")
  year, doy = int(date[:4]), int(date[4:7])
  date = str((datetime(year, 1, 1) + timedelta(doy - 1)).date())

  data_dict.setdefault(catalogue, {}).setdefault(layer, {}).setdefault(date, {})

  output_name = MODIS_PARAMS[catalogue]["output_name"]
  df = load_modis_tif(data_path, catalogue, date)
  if output_name == "LST_Celsius":
    all_lst_dfs.append(df)
  elif output_name == "LandCover_Type":
    all_landcover_dfs.append(df)
  elif output_name == "NDVI_Value":
    all_ndvi_dfs.append(df)
  elif output_name == "NPP_kgCm2perYear":
    all_npp_dfs.append(df)
  elif output_name == "TreeCover_Percent":
    all_treecover_dfs.append(df)

all_dfs = {"LST": all_lst_dfs, "LandCover": all_landcover_dfs, "NDVI": all_ndvi_dfs, "NPP": all_npp_dfs, "TreeCover": all_treecover_dfs}

for k in all_dfs:
  if all_dfs[k]:
    full_df = pd.concat(all_dfs[k], ignore_index=True)
    full_df.to_csv(f"all_{k}_data.csv", index=False)

In [ ]:
csv_file = 'formulization_data_final.csv'

if not os.path.exists(csv_file):
    print(f"ERROR: File '{csv_file}' not found!")
    exit()
try:
    df = pd.read_csv(csv_file)
    print(f"Dataset loaded successfully. Total rows: {len(df)}")
except Exception as e:
    print(f"ERROR: An error occurred while reading the file: {e}")
    exit()

required_cols = ['LandCover_Type', 'TreeCover_Percent', 'LST_Celsius', 'NDVI_Value', 'NPP_kgCm2perYear', 'latitude', 'longitude']
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    print(f"ERROR: Missing columns: {missing_cols}")
    exit()

Dataset loaded successfully. Total rows: 180480


# Normalization and Methodology

In [ ]:
suitable_land_types = [10, 12, 14]

is_suitable_land = df['LandCover_Type'].isin(suitable_land_types)
is_low_tree_cover = df['TreeCover_Percent'] < 60
has_no_nan = df[required_cols].notna().all(axis=1)

suitable_mask = is_suitable_land & is_low_tree_cover & has_no_nan

print(f"Number of suitable points found to be scored: {suitable_mask.sum()}")

df['suitability_score'] = 0.0

if suitable_mask.sum() > 0:
    # Normalization
    norm_cols = ['LST_Celsius', 'NDVI_Value', 'NPP_kgCm2perYear', 'TreeCover_Percent']

    for col in norm_cols:
        min_val = df.loc[suitable_mask, col].min()
        max_val = df.loc[suitable_mask, col].max()

        if (max_val - min_val) > 0:
            df.loc[suitable_mask, f'{col}_norm'] = (df.loc[suitable_mask, col] - min_val) / (max_val - min_val)
        else:
            df.loc[suitable_mask, f'{col}_norm'] = 0.5
            print(f"Warning: All values for {col} are the same, a value of 0.5 has been assigned.")

    # Methodology
    weights = {
        'LST': 0.40,
        'NDVI': 0.20,
        'NPP': 0.20,
        'TreeCover': 0.20
    }

    df.loc[suitable_mask, 'suitability_score'] = (
        weights['LST'] * df.loc[suitable_mask, 'LST_Celsius_norm'] +
        weights['NDVI'] * df.loc[suitable_mask, 'NDVI_Value_norm'] +
        weights['NPP'] * df.loc[suitable_mask, 'NPP_kgCm2perYear_norm'] +
        weights['TreeCover'] * (1 - df.loc[suitable_mask, 'TreeCover_Percent_norm'])
    )


results = df.sort_values(by='suitability_score', ascending=False)

Number of suitable points found to be scored: 58740


# Final

In [ ]:
output_cols = ['latitude', 'longitude', 'suitability_score']
if 'date' in df.columns or 'Date' in df.columns:
    date_col = 'date' if 'date' in df.columns else 'Date'
    results.rename(columns={date_col: 'date'}, inplace=True)
    output_cols = ['date'] + output_cols

final_locations = results[output_cols].copy()

final_locations['suitability_score'] = (final_locations['suitability_score'] * 100).round(2)

print(f"Top {min(10, len(final_locations))} most suitable locations for tree planting:")
print(final_locations.head(10))

print("Some example locations with a score of 0:")
print(final_locations[final_locations['suitability_score'] == 0].head(5))

output_file = 'tree_analysis.csv'
try:
    final_locations.to_csv(output_file, index=False, encoding='utf-8')

    print(f"Results have been saved to '{output_file}'.")
    print(f"  A total of {len(final_locations)} points were written to the file (including unsuitable ones).")
except Exception as e:
    print(f"ERROR: Could not save the CSV file: {e}")

Top 10 most suitable locations for tree planting:
              date  latitude  longitude  suitability_score
101244  2024-07-19     40.73       30.2              89.59
97297   2024-07-11     40.73       30.2              89.23
113087  2024-08-12     40.73       30.2              88.97
117034  2024-08-20     40.73       30.2              88.91
93349   2024-07-03     40.73       30.2              88.04
105192  2024-07-27     40.73       30.2              86.89
85453   2024-06-17     40.73       30.2              86.34
81505   2024-06-09     40.73       30.2              86.24
77557   2024-06-01     40.73       30.2              86.17
120981  2024-08-28     40.73       30.2              85.91
Some example locations with a score of 0:
              date  latitude  longitude  suitability_score
134307  2024-09-29     40.85      29.71                0.0
134306  2024-09-29     40.84      29.71                0.0
2485    2024-01-01     40.64      30.00                0.0
129434  2024-09-21     